In [32]:
import pandas as pd
import numpy as np

In [33]:
with pd.ExcelFile('input/input_pkmn_stats_and_evolutions.xlsx') as xl:
    stats = pd.read_excel(xl, sheet_name='pkmn_stats')
    evol = pd.read_excel(xl, sheet_name='pkmn_evolutions')

In [34]:
#From pkmn_stats dataset remove the columns height, weight and evolves from
stats.drop(columns=['height','weight',"evolves_from"], inplace=True)

In [35]:
#Pivot (wide to long) pkmn stats so that hp, attack, defense, special_attack, special_defense, 
#and speed become a column called 'combat_factors'
stats_pivot = pd.melt(stats, id_vars=['name','pokedex_number','gen_introduced'], var_name='combat_factors')

In [36]:
stats_pivot.head()

,name,pokedex_number,gen_introduced,combat_factors,value
0,Bulbasaur,1,1,hp,45
1,Ivysaur,2,1,hp,60
2,Venusaur,3,1,hp,80
3,Charmander,4,1,hp,39
4,Charmeleon,5,1,hp,58


In [37]:
#Find the combat power values relating to the Pokémon's last evolution stage
#Sum together each Pokémon's combat_factors
stats_pivot['combat_power'] = stats_pivot.groupby('name')['value'].transform('sum')

In [38]:
stats_pivot.head()

,name,pokedex_number,gen_introduced,combat_factors,value,combat_power
0,Bulbasaur,1,1,hp,45,318
1,Ivysaur,2,1,hp,60,405
2,Venusaur,3,1,hp,80,525
3,Charmander,4,1,hp,39,309
4,Charmeleon,5,1,hp,58,405


In [39]:
power_dict = dict(zip(stats_pivot['name'], stats_pivot["combat_power"]))

In [40]:
evol.head()

,Stage_1,Stage_2,Stage_3
0,Bulbasaur,Ivysaur,Venusaur
1,Charmander,Charmeleon,Charizard
2,Squirtle,Wartortle,Blastoise
3,Caterpie,Metapod,Butterfree
4,Weedle,Kakuna,Beedrill


In [41]:
evol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Stage_1  493 non-null    object
 1   Stage_2  330 non-null    object
 2   Stage_3  110 non-null    object
dtypes: object(3)
memory usage: 11.7+ KB


In [42]:
evol.dropna(subset=['Stage_2'], inplace=True)

In [43]:
#Using the evolutions data look up the combat_factors for each Pokémon at each stage, 
#making sure that the combat_factors match across the row, 
#i.e. we should be able to see the hp for Bulbasaur, Ivysaur and Venusaur on one row
df = pd.merge(evol, stats_pivot, how='left', left_on='Stage_1', right_on='name')\
    .drop(columns=['name',"combat_factors","value"])\
    .rename(columns={"combat_power":"initial_combat_power"})

In [44]:
df.drop_duplicates(inplace=True)

In [45]:
df.head()

,Stage_1,Stage_2,Stage_3,pokedex_number,gen_introduced,initial_combat_power
0,Bulbasaur,Ivysaur,Venusaur,1,1,318
6,Charmander,Charmeleon,Charizard,4,1,309
12,Squirtle,Wartortle,Blastoise,7,1,314
18,Caterpie,Metapod,Butterfree,10,1,195
24,Weedle,Kakuna,Beedrill,13,1,195


In [46]:
df['stage2evo'] = df['Stage_2'].map(power_dict)
df['stage3evo'] = df['Stage_3'].map(power_dict)
df['final_combat_power'] = np.where(pd.isnull(df['Stage_3']), df['stage2evo'], df['stage3evo'])

In [47]:
df.drop(columns=['stage2evo', 'stage3evo'], inplace=True)

In [48]:
df.head()

,Stage_1,Stage_2,Stage_3,pokedex_number,gen_introduced,initial_combat_power,final_combat_power
0,Bulbasaur,Ivysaur,Venusaur,1,1,318,525.0
6,Charmander,Charmeleon,Charizard,4,1,309,534.0
12,Squirtle,Wartortle,Blastoise,7,1,314,530.0
18,Caterpie,Metapod,Butterfree,10,1,195,395.0
24,Weedle,Kakuna,Beedrill,13,1,195,395.0


In [49]:
df['combat_power_increase'] = (df['final_combat_power'] - df['initial_combat_power']) / df['initial_combat_power']

df.sort_values('combat_power_increase', inplace=True)

df = df[["Stage_1","Stage_2","Stage_3","pokedex_number","gen_introduced",\
         "initial_combat_power","final_combat_power"]]

In [50]:
df.to_csv('output/output-2022-week08.csv', index=False)

In [51]:
output = pd.read_csv('output/output-2022-week08.csv')
output.head()

,Stage_1,Stage_2,Stage_3,pokedex_number,gen_introduced,initial_combat_power,final_combat_power
0,Nincada,Shedinja,NaN,290,3,266,236.0
1,Scyther,Scizor,NaN,123,1,500,500.0
2,Scyther,Kleavor,NaN,123,1,500,500.0
3,Type: Null,Silvally,NaN,772,7,534,570.0
4,Stantler,Wyrdeer,NaN,234,2,465,525.0
